<a href="https://colab.research.google.com/github/bilmark0/Agile-Manufacturing-TDK-/blob/main/CV/CNN_without_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import random
from google.colab import drive
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import PIL
import PIL.Image
import sys

In [ ]:
"""
# Clone the GitHub repository
!git clone https://github.com/bilmark0/Agile-Manufacturing-TDK-.git

# Append the modules path
sys.path.append("/content/Agile-Manufacturing-TDK-/CV/modules")

# Import DependencyManager from the correct module path
from dependency_manager import DependencyManager
"""

In [ ]:
!pip install kaggle
from google.colab import files
import zipfile

uploaded = files.upload()

# Move the uploaded kaggle.json file to the proper directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the training data from kaggle
!kaggle competitions download -c airbus-ship-detection

# Unzip the file and delete the zip afterwards
zip_file = '/content/airbus-ship-detection.zip'
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall()
os.remove(zip_file)
if os.path.exists('/content/sample_data'):
  shutil.rmtree('/content/sample_data')

print("Download and extraction complete.")

In [ ]:
data_dir = '/path/to/your/faulty_directory'
img_height, img_width = 240, 380

# Data generator with augmentation and validation split
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.1  # Use 10% of the data for validation
)

# Training data generator
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='binary',
    subset='training'  # Set as training data
)

# Validation data generator
val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # Set as validation data
)

# Build the Custom CNN Model
model = Sequential()

# Convolutional Block 1
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

# Convolutional Block 2
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

# Convolutional Block 3
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

# Convolutional Block 4
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

# Flatten and Fully Connected Layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))  # Dense layer with 512 neurons
model.add(Dropout(0.5))  # Dropout to prevent overfitting
model.add(Dense(256, activation='relu'))  # Dense layer with 256 neurons
model.add(Dropout(0.5))  # Additional dropout
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model with a binary cross-entropy loss function for binary classification
model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    epochs=100,  # Increased epochs to take advantage of larger dataset
    validation_data=val_generator,
    callbacks=[early_stopping]
)

# Evaluate the model
loss, accuracy = model.evaluate(val_generator)
print(f'Validation Loss: {loss}, Accuracy: {accuracy}')

# Save the trained model
model.save('/content/drive/MyDrive/object_detection_custom_cnn_model.h5')